#  Scipyによる最適化 -第3回-

Scientific computing tools or Python

https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html

SciPyは，Pythonによる科学技術計算のためのオープンソースソフトウェアを利用可能な仕組みである。

Scipyは，NumPyパッケージを内部で用いている。具体的には，NumPyで定義された基本的なデータ構造（配列や行列）や，基本的な演算を用いている。また，図示するためにはMatplotlibを用いている。


# 制約付き最適化（多変数スカラー関数）


`minimize`は，制約付き最適化問題を解くこともできる。ここでは，`trust-constr` を用いる方法を学ぶ。

ここで扱う問題は，次の制約付き最適化問題である。

$$ 
\text{最小化}_{x_0,x_1}  \ \ 100\left(x_1-x_0^2 \right)^2 + \left(1-x_0\right)^2 
$$

$$
\begin{array}{ll}
\text{制約条件} &  x_0+2x_1 \leq 1 \\
& x_0^2+x_1 \leq 1 \\ 
& x_0^2-x_1 \leq 1 \\ 
& 2x_0+x_1 = 1 \\ 
& 0 \leq x_0 \leq 1 \\ 
& -0.5 \leq x_1 \leq 2.0
\end{array}
$$

`trust-constr`を用いることで，信頼領域制約アルゴリズム(Trust-Region Constrained Algorithm)を実行することができる。信頼領域制約アルゴリズムでは，次の最適化問題を解くことができる。

$$
\begin{array}{ll}
\text{最小化}_x & f(x) \\
\text{制約条件} &  c^{\ell} \leq c(x) \leq c^u \\
& x^{\ell} \leq x \leq x^u.
\end{array}
$$


$c^{\ell}$はベクトルである。その$j$番目の要素$c^{\ell}_j$について$c^{\ell}_j=c^u_j$であれば，$j$番目の制約は等式制約であることを指定する。

## 上下限制約

上下限制約$0 \leq x_0 \leq 1$と$-0.5 \leq x_1 \leq 2.0$は，`Bounds`オブジェクトを用いて次のように定義できる。

In [ ]:
from scipy.optimize import Bounds 
bounds = Bounds([0,-0.5],[1.0,2.0])

## 線形制約
線形制約 $x_0+2x_1 \leq 1$と$2x_0+x_1=1$は，次の標準形で行列を用いて表される。　

$$
\begin{bmatrix}
-\infty \\ 
1
\end{bmatrix}
\leq
\begin{bmatrix}
1 & 2 \\ 
2 & 1 
\end{bmatrix}
\begin{bmatrix}
x_0 \\ 
x_1 
\end{bmatrix}
\leq 
\begin{bmatrix}
1 \\
1
\end{bmatrix}.
$$
これら２つの線形制約を表すプログラムは，次のとおりである。


In [ ]:
from scipy.optimize import LinearConstraint 
linear_constraint = LinearConstraint([[1,2],[2,1]],[-np.inf,1],[1,1]) 

## 非線形制約

線形制約の場合と同様に，2つの非線形制約は，次の形式で表される。

$$
c(x)
=
\begin{bmatrix}
c_0(x) \\
c_1(x)
\end{bmatrix}
=
\begin{bmatrix}
x_0^2 + x_1 \\ 
x_0^2 - x_1
\end{bmatrix}
\leq 
\begin{bmatrix}
1 \\ 
1
\end{bmatrix}
$$

ここで定めた関数$c(x)$ のヤコビ行列$J(x)$は次のものとなる。
$$
J(x) = 
\begin{bmatrix}
\frac{\partial c_0}{\partial x_0} & \frac{\partial c_0}{\partial x_1} \\ 
\frac{\partial c_1}{\partial x_0} & \frac{\partial c_1}{\partial x_1}
\end{bmatrix}
=
\begin{bmatrix}
2x_0 & 1 \\ 
2x_0 & -1
\end{bmatrix}
$$

また，$c_0(x)$のヘッセ行列$\nabla^2 c_0$と$c_1(x)$のヘッセ行列$\nabla^2 c_1$は，それぞれ

$$
\nabla^2 c_0=
\begin{bmatrix}
\frac{\partial^2 c_0}{\partial^2 x_0} & \frac{\partial^2 c_0}{\partial x_0 \partial x_1} \\  
\frac{\partial^2 c_0}{\partial x_1 \partial x_0} & \frac{\partial^2 c_0}{\partial x_1^2}
\end{bmatrix}
=
\begin{bmatrix}
 2&0  \\  
 0&0
\end{bmatrix},
\nabla^2 c_1=
\begin{bmatrix}
\frac{\partial^2 c_1}{\partial^2 x_0} & \frac{\partial^2 c_1}{\partial x_0 \partial x_1} \\  
\frac{\partial^2 c_1}{\partial x_1 \partial x_0} & \frac{\partial^2 c_1}{\partial x_1^2}
\end{bmatrix}
=
\begin{bmatrix}
 2&0  \\  
 0&0
\end{bmatrix},
$$
となるので，それらの線形結合は
$$
H(x,v)=\sum_{i=0}^1 v_i \nabla^2 c_i(x) = 
v_0 
\begin{bmatrix}
2 & 0 \\
0 & 0
\end{bmatrix}
+v_1
\begin{bmatrix}
2 & 0 \\ 
0 & 0
\end{bmatrix}
$$
と表される。この制約は，`NonlinearConstraint`オブジェクトを用いて次のように表すことができる。

In [ ]:
def cons_f(x):
    return [x[0]**2+x[1],x[0]**2-x[1]]

def cons_J(x):
    return [[2*x[0],1],[2*x[0],-1]]

def cons_H(x,v):
    return v[0]*np.array([[2,0],[0,0]])+v[1]*np.array([[2,0],[0,0]])

In [ ]:
from scipy.optimize import NonlinearConstraint 
nonlinear_constraint = NonlinearConstraint(cons_f,-np.inf,1,jac=cons_J,hess=cons_H)

`NonlinearConstraint` の最初の引数として指定している`cons_f`は，$c_1(x)$と$c_2(x)$を返り値とする関数(`cons_f`)です。二番目の引数は，$c_1(x)$と$c_2(x)$の下限を指定します。この問題では下限は指定されていないので，マイナス無限大としています。第1成分も第2成分も同じ値なので，`-np.inf`を1つだけ指定しています。3番目の引数は，上限を定めるものです。これも，第1成分と第2成分で同じ1なので，1つのスカラー値1を指定しています。`jac=`ではヤコビ行列を指定します。ヤコビ行列の指定には，`jac=`に続いてヤコビ行列を返す関数を与えます。ヤコビ行列は2x2行列なので，関数`jacob`は2x2の行列を返す関数です。行列は，2つの要素を持つ配列を2つの成分とする配列`[[2*x[0],1],[2*x[0],-1]]`で表されています。



## 最適化

こうして定義した要素を用いて，制約付き最適化問題を解くプログラムは，次の通りです。

In [ ]:
x0 = np.array([0.5, 0])
res = minimize(rosen, x0, method='trust-constr', jac=rosen_der, hess=rosen_hess,
               constraints=[linear_constraint, nonlinear_constraint],
               options={'verbose': 1}, bounds=bounds)
print(res.x)


In [ ]:
def cf(x):
    return 13*x[0]**2+10*x[0]*x[1]+7*x[1]**2+x[0]+x[1]+2

lc = LinearConstraint([[2,-5],[1,1]],[2,1],[2,1]) 

x0 = np.array([0.5, 0])
res = minimize(cf, x0, method='trust-constr',constraints=[lc],
               options={'verbose': 1}, bounds=bounds)
print(res.x)

--------

## 課題3

(1) 次の制約付き最小化問題を，`minimize`を用いて解け。

$$
\begin{array}{ll}
\text{最小化} & 13x_0^2+10x_0x_1+7x_1^2+x_0+x_1+2 \\
\text{制約条件} & 2x_0-5x_1 = 2 \\ 
& x_0+x_1 =1
\end{array}
$$

(2) 次の制約付き最小化問題を，`minimize`を用いて解け。

$$
\begin{array}{ll}
\text{最小化} & 13x_0^2+10x_0x_1+7x_1^2+x_0+x_1+2 \\
\text{制約条件} & 2\sin(x_0)-5x_1 \leq 2 \\ 
& x_0+x_1 =1
\end{array}
$$


--------


In [ ]:
##ここにプログラムを作成してください

#def cf(x):
#    ...    

#def cons_f1(x):
#    ...
    
#def cons_J1(x):
#    ...

#def cons_H1(x,v):
#    ... 
    
#nlcst = NonlinearConstraint(...)

#res = minimize(... )

#print(res.x)

# 大域的最適化 

ここまで述べたアルゴリズムでは，大域的最適解ではなく，局所最適解が得られる。局所最適解を求めるアルゴリズムは，一般的に，初期解から始めてよりよい解を反復的に探索し，それ以上（局所的に）改善できなくなったらアルゴリズムを終了する。それに対して，ここでは，大域的最適解を求めようとするアルゴリズムを扱う。大域的最適解を求めようとするアルゴリズムも，初期解から始めてよりよい解を探索するが，局所的に改善できない解（局所最適解）に到達しても，そこからさらによりよい解を求めようとする。このとき，局所最適解から抜け出してさらに解の探索を進める方法の違いにより，様々なアルゴリズムが得られる。ただし，それらのアルゴリズムの実行が終了しても，必ずしも最適解が得られているとは限らない。

SciPyは，大域的最適解を求めるアルゴリズムをいくつか提供している。ここでは，次で定義される`eggholder`関数を扱う。

$$
f(x)=\left(
-\left(x_1+47\right)) \sin \left( \sqrt{\left| x_0/2+\left(x_1+47\right)\right|} \right) 
-x_0 \sin\left( \sqrt{|x_0-\left(x_1+47\right)|} \right)
   \right)
$$


この`eggholder`関数をPythonで定義するには，次のようにする。

In [ ]:
def eggholder(x):
    return (-(x[1] + 47) * np.sin(np.sqrt(abs(x[0]/2 + (x[1]  + 47))))\
            -x[0] * np.sin(np.sqrt(abs(x[0] - (x[1]  + 47)))))

bounds = [(-512, 512), (-512, 512)]

この関数を，`matplotlib`の機能を用いて図示する。まず，必要な機能を`import`する。

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

`x`を，-512から513までの範囲で値をとる数，`y`を，-512から513までの範囲で値をとる数とする。さらに，`meshgrid`関数を用いて，`x`と`y`の値のペアを生成する。

In [ ]:
x = np.arange(-512,513)
y = np.arange(-512,513)
xgrid, ygrid = np.meshgrid(x,y)
xy = np.stack([xgrid,ygrid])

各(x,y)の値における`eggholder`関数の値を，`plot_surface`関数を用いてプロットする。

In [ ]:
fig = plt.figure()
ax=fig.add_subplot(111,projection='3d')
ax.view_init(45,-45)
ax.plot_surface(xgrid,ygrid,eggholder(xy),cmap='terrain')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('eggholder(x,y)')
plt.show()

プロットの結果からわかるように，この関数には局所最適解が多数ある。

次に，この`eggholder`関数の最小解を，異なるアルゴリズムによって計算する。複数のアルゴリズムの実行結果を保存して比較したいので，計算結果を保存する辞書`results`を用意する。まず，`optimize.shgo`による計算結果を，`result['shgo']`に保管する。 `optimize.shgo`は，simplicial homology global optimizationを実行するものである。

In [ ]:
from scipy import optimize 
results = dict()
results['shgo']=optimize.shgo(eggholder,bounds)
results['shgo']

これより，得られた最適値は，-935.34であることがわかる。

次に，Dual Anealing法により最適解を得ようとする。



In [ ]:
results['DA'] = optimize.dual_annealing(eggholder, bounds)
results['DA']

このように，`shgo`の場合よりも目的関数値の小さい解が得られている。このことは，`shgo`で得られた解が大域的最適解ではないことを示している。

--------

## 課題4

差分進化(differential evolution)とBasin Hoppingにより，`eggholder`関数を最小化せよ。得られた結果は，それぞれ`result['DE']`, `result['BH']`に保存せよ。さらに，これらの得られた結果と`result['shgo'], result['DA']`を比較せよ。

--------


In [ ]:
##ここにプログラムを作成してください

#results['DE'] = optimize.....
#results['BH'] = optimize.....

-----------

# 最適化技術実験 第3回　レポート
### 学生番号 000000 氏名 青学太郎

(ここから下に，レポートを作成してください。レポートを作成したら，「File」->「Save and Checkpoint」でこのjupyter notebookファイルを保存し，さらに，「File」->「Download as」->「Notebook(.ipynb)」を選択して手元のPCにダウンロードしてください．ダウンロードしたファイルの名前を「学生番号-氏名.ipynb」に変更して，CoursePowerから提出してください．ここで，ファイル名内の"学生番号"は1から始まる自分の学生番号に，"氏名"は自分の氏名に置き換えてください．）